In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json

# scrapper

In [3]:
url1 = 'http://www.markedbyteachers.com/catalogsearch/result/index/?dir=desc&order=marked_by_teacher&p=1'
res1 = requests.get(url1)

soup1 = BeautifulSoup(res1.text)

pages_div = soup1.find('div',{'class':'pages'})

pages_num = re.findall('\d+',pages_div.find('label').getText())
pages_num = int(pages_num[0])
cnt = 0



base_url = 'http://www.markedbyteachers.com/catalogsearch/result/index/?dir=desc&order=marked_by_teacher&p={}'
for p in range(1,pages_num+1):
    try:
        res = requests.get(base_url.format(str(p)))
        soup = BeautifulSoup(res.text)
        
        essays_div = soup.find('div',{'class':'category-products'})
        essays = essays_div.findAll('li',{'class':'product'})
        
        for essay in essays:
            try:
                essay_id = str(essay['id'])
            
                essay_title = essay.find("h2",{'class':'product-name'}).getText(strip=True)
                rate = essay.find('div',{'class':'review-stars'}).getText(strip=True)

                info_lis = essay.find('ul',{'class':'product-info'}).findAll('li')

                level = info_lis[0].getText(strip=True).replace('Level:\r\n                        ','')
                subject = info_lis[1].getText(strip=True).replace('Subject:\r\n                        ','')
                word_count = int(info_lis[2].getText(strip=True).replace('Word count:\r\n                        ',''))
                submitted = info_lis[3].getText(strip=True).replace('Submitted:\r\n                        ','')
                marked_by_teacher = {'teacher_name':info_lis[4].findAll('a')[1].getText(strip=True),'date':info_lis[4].findAll('a')[1].nextSibling.strip()}

                essay_url = essay.find('ul',{'class':'product-actions'}).find('li').find('a')['href']


                record = {
                    'essay_id':essay_id,
                    'essay_title':essay_title,
                    'rate':rate,
                    'level':level,
                    'subject':subject,
                    'word_count':word_count,
                    'submitted':submitted,
                    'marked_by_teacher':marked_by_teacher,
                    'essay_url':essay_url
                }
                
                with open('data/{}.json'.format(essay_id) ,'w') as f:
                    json.dump(record ,f)
                    
                cnt += 1
                    
                
                

            except Exception as e:
                print('record error {} '.format(  e))
        print('finished page : {}  ,  records : {}'.format(str(p) ,str(cnt)))

    except Exception as e:
        print("page error",e)
        
        
        
        
    
    


record error 'NoneType' object has no attribute 'getText' 
finished page : 1  ,  records : 9
record error 'NoneType' object has no attribute 'getText' 
record error 'NoneType' object has no attribute 'getText' 
finished page : 2  ,  records : 17
record error 'NoneType' object has no attribute 'getText' 
finished page : 3  ,  records : 26
finished page : 4  ,  records : 36
record error 'NoneType' object has no attribute 'getText' 
finished page : 5  ,  records : 45
finished page : 6  ,  records : 55
record error 'NoneType' object has no attribute 'getText' 
record error 'NoneType' object has no attribute 'getText' 
finished page : 7  ,  records : 63
finished page : 8  ,  records : 73
record error 'NoneType' object has no attribute 'getText' 
finished page : 9  ,  records : 82
record error 'NoneType' object has no attribute 'getText' 
finished page : 10  ,  records : 91
record error 'NoneType' object has no attribute 'getText' 
finished page : 11  ,  records : 100
finished page : 12  ,  

record error 'NoneType' object has no attribute 'getText' 
finished page : 94  ,  records : 840
finished page : 95  ,  records : 850
record error 'NoneType' object has no attribute 'getText' 
record error 'NoneType' object has no attribute 'getText' 
finished page : 96  ,  records : 858
record error 'NoneType' object has no attribute 'getText' 
finished page : 97  ,  records : 867
record error 'NoneType' object has no attribute 'getText' 
finished page : 98  ,  records : 876
finished page : 99  ,  records : 886
finished page : 100  ,  records : 896
finished page : 101  ,  records : 906
finished page : 102  ,  records : 915
finished page : 103  ,  records : 924
record error 'NoneType' object has no attribute 'getText' 
record error invalid literal for int() with base 10: 'Submitted:\r\n                        26/09/2008' 
record error 'NoneType' object has no attribute 'getText' 
finished page : 104  ,  records : 931
record error 'NoneType' object has no attribute 'getText' 
record erro

finished page : 202  ,  records : 1845
record error 'NoneType' object has no attribute 'getText' 
finished page : 203  ,  records : 1854
record error 'NoneType' object has no attribute 'getText' 
finished page : 204  ,  records : 1863
record error 'NoneType' object has no attribute 'getText' 
finished page : 205  ,  records : 1872
finished page : 206  ,  records : 1882
record error 'NoneType' object has no attribute 'getText' 
finished page : 207  ,  records : 1891
finished page : 208  ,  records : 1901
record error 'NoneType' object has no attribute 'getText' 
finished page : 209  ,  records : 1910
finished page : 210  ,  records : 1920
record error 'NoneType' object has no attribute 'getText' 
finished page : 211  ,  records : 1929
finished page : 212  ,  records : 1939
finished page : 213  ,  records : 1949
finished page : 214  ,  records : 1959
record error 'NoneType' object has no attribute 'getText' 
record error 'NoneType' object has no attribute 'getText' 
finished page : 215  

KeyboardInterrupt: 

# stats

In [9]:
import os
files = os.listdir('data')
len(files)-1

2211

In [8]:
names = []
ids = []
urls = []
for file in files:
    if file.endswith('.json'):
        with open("data/"+file , 'r') as f:
            data = json.load(f)
            names.append(data['essay_title'])
            urls.append(data['essay_url'])
            
print('unique names : {}'.format(str(len(set(names)))))
print('unique urls : {}'.format(str(len(set(urls)))))



unique names : 2194
unique urls : 2211


In [13]:
print("expected patents : ",int(2211 * 161922 / 2420))

expected patents :  147937
